In [1]:
%load_ext autoreload
%autoreload 2
import sys
sys.path.append('../src')

In [2]:
from get_data import load_history, load_current, combine_history, process_y, load_bios, load_team_data
# load_history()
# load_current()
# combine_history()
# load_team_data()
# load_bios()
# process_y()


In [3]:
from process_data import get_rest_of_season_player_stats, PRED_COLS
from model_training import get_data_by_windows, get_simple_pipelines, load_player_feature_map

In [ ]:
force_retrain = True

player_type = 'goalie'
X_g, y_g, g_feature_map = get_data_by_windows(player_type, [10, 8, 3], force_retrain=force_retrain)
pipelines = get_simple_pipelines(player_type, (X_g, y_g.loc[X_g.index]), g_feature_map, force_retrain=force_retrain)

player_type = 'skater'
X, y, full_feature_map = get_data_by_windows(player_type, [10, 6, 3], force_retrain=force_retrain)
pipelines = get_simple_pipelines(player_type, (X, y.loc[X.index]), full_feature_map, force_retrain=force_retrain)


File not found. Fitting...
Fitting for ga
0 / 33
Selected feature xPlayContinuedInZone with score 0.33903038049398626
0 / 29
Selected feature goals with score 0.3455347474426962
0 / 11
Selected feature mediumDangerGoals with score 0.34590700515354306
Fitting for win
0 / 33
Selected feature xPlayStopped with score 0.2887499451809577
0 / 29
Selected feature win with score 0.308390591838034
0 / 8
Selected feature lowDangerGoals with score 0.31064513373617314
0 / 4
Selected feature blocked_shot_attempts with score 0.311018022836201
0 / 1
Fitting for so
0 / 33
Selected feature xPlayStopped with score 0.08401693575329823
0 / 29
Selected feature goals with score 0.0925056874129246
0 / 4
Selected feature win with score 0.09437964639383656
0 / 1
Selected feature lowDangerGoals with score 0.09463818365325449
Fitting for save
0 / 33
Selected feature xPlayContinuedOutsideZone with score 0.39929639555907204
0 / 29
Selected feature icetime with score 0.4001435309554076
0 / 4
Selected feature rebound

/opt/conda/envs/yh/lib/python3.12/site-packages/sklearn/linear_model/_coordinate_descent.py:697: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 1.283e+05, tolerance: 8.863e+01
  model = cd_fast.enet_coordinate_descent(
/opt/conda/envs/yh/lib/python3.12/site-packages/sklearn/linear_model/_coordinate_descent.py:697: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 1.740e+05, tolerance: 1.129e+02
  model = cd_fast.enet_coordinate_descent(


fitting icetime
icetime -- 0.22347294864951894
icetime -- 0.14310943851385233
File not found. Fitting...
Fitting for g
0 / 725
100 / 725
200 / 725
300 / 725
400 / 725
500 / 725
600 / 725
700 / 725
Selected feature all_I_F_xGoals_with_earned_rebounds_scoreAdjusted with score 0.41213160432748275
0 / 427
100 / 427
200 / 427
300 / 427
400 / 427
Selected feature all_I_F_points with score 0.4379700682706697
0 / 379
100 / 379
200 / 379
300 / 379
Selected feature 5on4_I_F_xPlayContinuedInZone with score 0.4469259699787799
0 / 305
100 / 305
200 / 305
300 / 305
Selected feature 5on5_corsiForAfterShifts with score 0.4571352311276476
0 / 189
100 / 189
Selected feature 5on5_I_F_goals with score 0.4637837205342761
0 / 155
100 / 155
Selected feature all_I_F_oZoneShiftStarts with score 0.46894989328722314
0 / 131
100 / 131
Selected feature all_I_F_lowDangerxGoals with score 0.470805603020116
0 / 72
Selected feature all_I_F_mediumDangerShots with score 0.4737925399915915
0 / 49
Selected feature all_I_F

In [ ]:
retrieved data for window 30
retrieved data for window 5
retrieved data for window 1
fitting ga
ga -- 0.2781581204561938
ga -- 0.4051276765714114
fitting win
win -- 0.3272036204679637
win -- 0.07062347607719272
fitting so
so -- 0.06548014483164122
so -- 0.014297566297834016
fitting save
save -- 0.33045140128618333
save -- 3.467425085824501
fitting icetime
icetime -- 0.3455776541663804
icetime -- 0.12820161273975547


retrieved data for window 30
retrieved data for window 5
fitting g
g -- 0.4089512808698992
g -- 0.06928785622807117
fitting a
a -- 0.489455453911782
a -- 0.09122309223345175
fitting sog
sog -- 0.7133130336579327
sog -- 0.2660641974523548
fitting fow
fow -- 0.8665709975279869
fow -- 0.3397963717803079
fitting hit
hit -- 0.7355184307746057
hit -- 0.290320711797206
fitting block
block -- 0.7538360461059199
block -- 0.1856533732504311
fitting pim
pim -- 0.32386333214189067
pim -- 0.16999748393168024
fitting goalsfor
goalsfor -- 0.34739104680856125
goalsfor -- 0.13793761773340132
fitting goalsaga
goalsaga -- 0.30290502725274415
goalsaga -- 0.13494602313267276
fitting ppp
ppp -- 0.5447973355464135
ppp -- 0.04284842462610222

In [ ]:
for col in PRED_COLS['skater']:    
    preds[col] = pipelines['skater'][col].predict(X[skaters_p_feats[col][0]])
preds_df = pd.DataFrame(preds, index=X.index)

gl_p_cols = list(set([c for s in goalies_p_feats.values() for c in s[0]]))
for col in PRED_COLS['goalie']:
    goalie_preds[col] = pipelines['goalie'][col].predict(X_g[goalies_p_feats[col][0]])
goalie_preds_df = pd.DataFrame(goalie_preds, index=X_g.index)



games_df = pd.DataFrame([dict(week=week, **game) for week, games in current_schedule.items() for game in games])

preds_df = preds_df.merge(games_df[['gameId','week']], on='gameId', how='left')\
    .set_index(preds_df.index)
goalie_preds_df = goalie_preds_df.merge(games_df[['gameId','week']], on='gameId', how='left')\
    .set_index(goalie_preds_df.index)

preds_df = pd.concat([preds_df, goalie_preds_df], axis=0)
weekly_preds = preds_df[preds_df['week'] > 0].set_index('week', append=True)
weekly_preds = weekly_preds.drop('gameId', axis=1)

weekly_preds.to_csv('data/2324_preds_simple.csv')

In [15]:
import pandas as pd
player_data = pd.read_csv('data/player_data.csv')
player_data.loc[player_data.pos != "['G']"].columns

Index(['playerId', 'name', 'pos', 'team', 'rank', 'week_rank', 'games',
       'week_games', 'g', 'a', 'sog', 'fow', 'hit', 'block', 'pim', 'plusmin',
       'ga', 'win', 'so', 'save', 'g_week', 'a_week', 'sog_week', 'fow_week',
       'hit_week', 'block_week', 'pim_week', 'plusmin_week', 'ga_week',
       'win_week', 'so_week', 'save_week', 'current_lineup', 'selection'],
      dtype='object')

In [ ]:
from yahoo_utils import get_games_by_week
current_schedule = get_games_by_week(427)

In [ ]:
preds = {}
goalie_preds = {}

for col in PRED_COLS['skater']:    
    preds[col] = pipelines['skater'][col].predict(skater_latest.dropna()[skaters_p_feats[col][0]])
preds_df = pd.DataFrame(preds, index=skater_latest.dropna().index)

gl_p_cols = list(set([c for s in goalies_p_feats.values() for c in s[0]]))
for col in PRED_COLS['goalie']:
    goalie_preds[col] = pipelines['goalie'][col].predict(goalie_latest.dropna()[goalies_p_feats[col][0]])
goalie_preds_df = pd.DataFrame(goalie_preds, index=goalie_latest.dropna().index)

preds_df = pd.concat([preds_df, goalie_preds_df], axis=0)
weekly_preds.to_csv('data/latest_preds.csv')